### Análise de ajustes

#### Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import t

#### Help functions

In [5]:
def first_analysis_lr(x, y, xlabel, ylabel):    
    #Cálculos e equação da reta entre duas variáveis aleatórias
    df = pd.DataFrame({'x':x, 'y':y})

    #x médio:
    x_mean = df['x'].mean()

    #y médio:
    y_mean = df['y'].mean()

    #Somatória dos valores de x:
    x_sum = df['x'].sum()

    #Somatória dos valores de y:
    y_sum = df['y'].sum()

    #Somatória de x * y:
    xy_sum = (df['x'] * df['y']).sum()

    #Somatória dos valores de x quadrado:
    x2_sum = (df['x']**2).sum()

    #Somatória dos valores de y quadrado:
    y2_sum = (df['y']**2).sum()

    #Número de amostras:
    n = df.shape[0]

    #Fórnulas complementares para cálculo de b1:
    sxy = xy_sum - ((x_sum * y_sum) / n)
    sxx = x2_sum - ((x_sum**2) / n)
    syy = y2_sum - ( (y_sum**2)/ n)

    #Estimativa do coeficiente de inclinação b1(beta 1):
    b1 = sxy / sxx

    #Estimativa do intercepto b_0(beta 0):
    b0 = (y_sum - (b1 * x_sum)) / n
    b0 = y_mean - (b1 * x_mean)

    #SQE => Soma de Quadrados dos Erros / SQT => Somatória de Quadrados Total / SQR => Soma de quadrados da Regressão:
    sqe = y2_sum - (b0 * y_sum) - (b1 * xy_sum)
    #alternativa: sqe = syy - (b1 * sxy)
    sqt = y2_sum - ( (y_sum**2)/ n)
    #alternativa: sqt = syy
    sqr = sqt - sqe

    #Equação da reta:
    print('Equação da reta: y = {} + {}x'.format(b0, b1))

    return {xlabel+'_mean':x_mean, 
            ylabel+'_mean':y_mean, 
            xlabel+'_sum':x_sum, 
            ylabel+'_sum':y_sum, 
            xlabel+'_'+ylabel+'_sum':xy_sum,
            xlabel+'2_sum':x2_sum, 
            ylabel+'2_sum':y2_sum,
            'n':n,
            'sxy':sxy,
            'sxx':sxx,
            'syy':syy,
            'b1':b1,
            'b0':b0,
            'sqe':sqe,
            'sqt':sqt,
            'sqr':sqr}



def var(sqe, n):
    #Variância(s2) e Desvio-padrão
    s2 = sqe / (n - 2)

    return {'variancia':s2}



def std(sqe, n):
    s2 = sqe / (n - 2)
    s = np.sqrt(s2)

    return {'std':s}



def coef_corr(sxy, sxx, syy):
    #Cálculo do coeficiente de correlação de Pearson
    r = sxy / np.sqrt(sxx*syy)

    return {'coef_corr':r}



def coef_det(sqe, sqt):
    #Coeficiente de Determinação => Explica se a reta descreve um modelo linear
    r2 = 1 - (sqe / sqt)

    return {'coef_det':r2}



def confidence_interval(s, sxx, ic, n, b1):
    #Intervalo de confiança:
    #Obter o erro padrão estimado
    sb1_inf = s / np.sqrt(sxx)
    print('Erro padrão estimado:', sb1_inf)

    #t student => t , alpha/2, n-2 (COnsultar na tabela)
    alpha = 1 - (ic / 100)
    gl = n - 2
    t_student = t.ppf(1 - (alpha / 2), gl)

    #b1_inf = b1 +- intervalo
    intervalo = t_student * sb1_inf

    print('IC de {}% = ({}, {})'.format(ic, b1 - intervalo, b1 + intervalo))

    return None

#### Dataset

In [6]:
df = pd.read_csv('data/carst.csv', delimiter = ',')
df.head()

,brand/model/year,cap_vol,consumo,power,weight,cemm,nu_cy,Etype
0,Gurgel BR800 0.8 1991,792,12.0,33,650,34.4,2,0
1,FIAT UNO Mille EP 1996,994,10.4,58,870,18.6,4,0
2,Hyundai HB20 Sense 2020,1000,12.8,80,989,14.5,3,0
3,FIAT Strada 1.4 2016,1368,10.3,86,1084,12.5,4,0
4,VolksWagen GOL 1.6 2015,1598,10.5,104,961,9.8,4,0


#### 1.
Encontrar os parâmetros e realizar a análise de ajustes de cap_vol (variável preditora) e consumo(variável resposta)

##### a. 
Obtenha a reta que descreve a relação entre as variáveis e o interrvalo de confiança para 95%, obtenha o valor esperado de y para x = 3000

In [7]:
x = df['cap_vol']
y = df['consumo']

first_analysis_lr(df['cap_vol'], df['consumo'], 'cap_vol', 'consumo')

x = 3000
b0 = 12.14
b1 = -0.0013
u_y3000 = b0 + (b1 * x)

print('Valor esperado de uy:3000 = {}'.format(u_y3000))

Equação da reta: y = 12.144010117932384 + -0.0013478460028317721x
Valor esperado de uy:3000 = 8.24


#### 3.
Observe a figura que representa um conjunto de dados bivariados, isto é, dados que apresentam informações semelhantes. Os dados, compostos por 20 amostras, estão bem correlacionados porque representam duas metodologias diferentes para obter a concentração de hidrogênio, esperando-se, portanto, que forneçam resultados semelhantes.
Com esses dados foi realizado o procedimento de regressão obtendo as seguintes informações: reta de regressão Y = -0.9625 + 1.001 x, uma variância estimada de 0,3022 e Sxx = 40925.75.
Calcule o intervalo de confiança de beta IC a 95%.

In [8]:
y = -0.9625 + (1.001 * x)
s2 = 0.3022
s = pow(s2, 1/2)
sxx = 40925.75
ic = 95
n = 20
b1 = 1.001
b0 = -0.9625

#Intervalo de confiança:
#Obter o erro padrão estimado
sb1_inf = s / pow(sxx, 1/2)
print('Erro padrão estimado:', sb1_inf)

#t student => t , alpha/2, n-2 (COnsultar na tabela)
alpha = 1 - (ic / 100)
gl = n - 2
t_student = t.ppf(1 - (alpha / 2), gl)

#b1_inf = b1 +- intervalo
intervalo = t_student * sb1_inf

print('IC de {}% = ({}, {})'.format(ic, b1 - intervalo, b1 + intervalo))

Erro padrão estimado: 0.0027173708138224666
IC de 95% = (0.9952910157657328, 1.0067089842342671)


#### 4. 
Um conjunto de dados fornece os seguintes valores: Sxx = 1585.2307, Syy = 77.2708, e Sxy = -341.9592, e uma reta de regressão Y = 9.10099 - 0.21572 x. Calcule a soma de quadrados dos erros e o coeficiente de determinação


In [25]:
sxx = 1585.2307
syy = 77.2708
sxy = -341.9592
y = 9.10099 - (0.21572 * x)
b1 = -0.21572
b0 = 9.10099

sqe = syy - (b1 * sxy)
print('SQE = {}'.format(sqe))

sqt = syy

coef = coef_det(sqe, sqt)
df['coef_det']
print('Coeficiente de determinação:{}'.format(coef['coef_det']))

SQE = 3.5033613759999866
Coeficiente de determinação:0.9546612513912114


#### 5.
Considere a seguinte amostra, na qual se apresenta a relação entre a taxa de eficiência de fosfatização de um peça de aço de acordo com a temperatura.

Temp.	76,67	 77,78	78,33	78,89	78,89	79,44	 80,00	 80,56	82,22	82,22	82,22	82,22<br>
Taxa	0,84	1,31	1,42	1,03	1,07	1,08	1,04	1,80	1,45	1,60	1,61	2,13<br>
<br>
Temp.	82,22	 82,78	82,78	 83,33	83,33	83,33	83,33	 84,44	84,44	85,00	85,56	86,67<br>
Taxa	2,15	0,84	1,43	0,90	1,81	1,94	2,68	1,49	2,52	3,0	1,87	3,08<br>

Aplique o procedimento de regressão linear para obter a reta que descreve esses dados, considere como variável preditora a Temperatura e como variável resposta a Taxa de eficiência, depois calcule os coeficientes de determinação, indique o que ele representa e calcule o valor esperado de Y para x= 80.

In [21]:
#valores
temp = [76.67, 77.78, 78.33, 78.89, 78.89, 79.44, 80.00, 80.56, 82.22, 82.22, 82.22, 82.22, 82.22, 82.78, 82.78, 83.33, 83.33, 83.33, 83.33, 84.44, 84.44, 85.00, 85.56, 86.67]
taxa = [0.84, 1.31, 1.42, 1.03, 1.07, 1.08, 1.04, 1.80, 1.45, 1.60, 1.61, 2.13, 2.15, 0.84, 1.43, 0.90, 1.81, 1.94, 2.68, 1.49, 2.52, 3.0, 1.87, 3.08]

#Análise de regressão linear
df = first_analysis_lr(temp, taxa, 'temp', 'taxa')

#valor esperado
x = 80
u_y80 = -12.232 + (0.17 * x)
print('Valor esperado de y para x = 80: {}'.format(u_y80))

#coeficiente de determinação
coef = coef_det(df['sqe'], df['sqt'])
print('Coeficiente de determinação:{}'.format(coef['coef_det']))

#Baixo coeficiente de determinação, o modelo não exemplifica uma regressão linear simples


Equação da reta: y = -12.231556513324294 + 0.16965263586290547x
Valor esperado de y para x = 80: 1.368000000000002
Coeficiente de determinação:0.4514004980380427
